In [1]:
import os, shutil, sys, time, re, requests, csv, datetime, pytz
import pandas as pd
import numpy as np
import netCDF4 as nc
import xarray as xr

/home/andrew/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
username = 'OOIAPI-C9OSZAQABG1H3U'
token = 'JA48WUQVG7F'

In [3]:
data_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv'
anno_url = 'https://ooinet.oceanobservatories.org/api/m2m/12580/anno/find'
vocab_url = 'https://ooinet.oceanobservatories.org/api/m2m/12586/vocab/inv'
asset_url = 'https://ooinet.oceanobservatories.org/api/m2m/12587'
deploy_url = asset_url + '/events/deployment/query'
cal_url = asset_url + '/asset/cal'

In [4]:
gross_range_filepath = '/home/andrew/Documents/OOI-CGSN/ooi-integration/qc-lookup/data_qc_global_range_values.csv'

In [5]:
# Function to make an API request 
def get_api(url):
    r = requests.get(url, auth=(username, token))
    data = r.json()
    return data

# Function to make an API request and print the results
def get_and_print_api(url):
    r = requests.get(url, auth=(username, token))
    data = r.json()
    for d in data:
        print(d)
        
# Specify some functions to convert timestamps
ntp_epoch = datetime.datetime(1900, 1, 1)
unix_epoch = datetime.datetime(1970, 1, 1)
ntp_delta = (unix_epoch - ntp_epoch).total_seconds()

def ntp_seconds_to_datetime(ntp_seconds):
    return datetime.datetime.utcfromtimestamp(ntp_seconds - ntp_delta).replace(microsecond=0)
  
def convert_time(ms):
    if ms is None:
        return None
    else:
        return datetime.datetime.utcfromtimestamp(ms/1000)

In [6]:
gross_range_lookup_table = pd.read_csv(gross_range_filepath)
gross_range_lookup_table.head()

,ReferenceDesignator,ParameterID_R,ParameterID_T,GlobalRangeMin,GlobalRangeMax,_DataLevel,_Units,_Array ID,_Platform ID,_Instrument
0,CE01ISSM-MFD35-01-VEL3DD000,vel3d_c_eastward_turbulent_velocity,vel3d_c_eastward_turbulent_velocity,-3.0,3.0,L1,m s-1,CE01ISSM,MFD35,VEL3DD000
1,CE01ISSM-MFD35-01-VEL3DD000,vel3d_c_northward_turbulent_velocity,vel3d_c_northward_turbulent_velocity,-3.0,3.0,L1,m s-1,CE01ISSM,MFD35,VEL3DD000
2,CE01ISSM-MFD35-01-VEL3DD000,vel3d_c_upward_turbulent_velocity,vel3d_c_upward_turbulent_velocity,-1.0,1.0,L1,m s-1,CE01ISSM,MFD35,VEL3DD000
3,CE01ISSM-MFD35-01-VEL3DD000,seawater_pressure_mbar,seawater_pressure_mbar,0.0,1100000.0,NaN,0.001 dbar,CE01ISSM,MFD35,VEL3DD000
4,CE01ISSM-MFD35-01-VEL3DD000,turbulent_velocity_east,turbulent_velocity_east,-3000.0,3000.0,L0,mm s-1,CE01ISSM,MFD35,VEL3DD000


In [30]:
# Filter out the non-WHOI responsible 
all_sites = get_api(data_url)
sites = []
for x in all_sites:
    if x.startswith('CE') or x.startswith('RS') or x.startswith('SS') or x.endswith('MOAS'):
        pass
    else:
        sites.append(x)
sites

['CP01CNPM',
 'CP01CNSM',
 'CP01CNSP',
 'CP02PMCI',
 'CP02PMCO',
 'CP02PMUI',
 'CP02PMUO',
 'CP03ISPM',
 'CP03ISSM',
 'CP03ISSP',
 'CP04OSPM',
 'CP04OSSM',
 'GA01SUMO',
 'GA02HYPM',
 'GA03FLMA',
 'GA03FLMB',
 'GI01SUMO',
 'GI02HYPM',
 'GI03FLMA',
 'GI03FLMB',
 'GP02HYPM',
 'GP03FLMA',
 'GP03FLMB',
 'GS01SUMO',
 'GS02HYPM',
 'GS03FLMA',
 'GS03FLMB']

In [76]:
df = pd.DataFrame(columns=['RefDes','Method','Stream'])

In [77]:
df

,RefDes,Method,Stream


In [82]:
refdes = []
pd.DataFrame(columns=['RefDes','Method','Stream'])

for site in sites:
    nodes = get_api('/'.join((data_url,site)))
    for node in nodes:
        sensors = get_api('/'.join((data_url,site,node)))
        for sensor in sensors:
            # Now check for the CTDBP in the system
            if 'CTDBP' in sensor:
                # Get the reference designator of the sensor
                request_url = '/'.join((vocab_url, site, node, sensor))
                r = requests.get(request_url, auth=(username, token))
                data = r.json()
                rfd = data[0]['refdes']
                refdes.append(rfd)
                # Now I'm going to 
                methods = get_api('/'.join((data_url,site,node,sensor)))
                for method in methods:
                    streams = get_api('/'.join((data_url,site,node,sensor,method)))
                    for stream in streams:
                        # Now update the dataframe
                        df['RefDes'].concat(rfd)
                        df['Method'].concat(method)
                        df['Stream'].concat(stream)
            else:
                pass            

AttributeError: 'Series' object has no attribute 'concat'

In [80]:
refdes

['CP01CNSM-MFD37-03-CTDBPD000',
 'CP01CNSM-RID27-03-CTDBPC000',
 'CP03ISSM-MFD37-03-CTDBPD000',
 'CP03ISSM-RID27-03-CTDBPC000',
 'CP04OSSM-MFD37-03-CTDBPE000',
 'CP04OSSM-RID27-03-CTDBPC000',
 'GA01SUMO-RID16-03-CTDBPF000',
 'GA01SUMO-RII11-02-CTDBPP031',
 'GA01SUMO-RII11-02-CTDBPP032',
 'GA01SUMO-RII11-02-CTDBPP033',
 'GI01SUMO-RID16-03-CTDBPF000',
 'GI01SUMO-RII11-02-CTDBPP031',
 'GI01SUMO-RII11-02-CTDBPP032',
 'GI01SUMO-RII11-02-CTDBPP033',
 'GS01SUMO-RID16-03-CTDBPF000',
 'GS01SUMO-RII11-02-CTDBPP031',
 'GS01SUMO-RII11-02-CTDBPP032',
 'GS01SUMO-RII11-02-CTDBPP033']

In [71]:
site = sites[1]
nodes = get_api('/'.join([data_url,site]))
node = nodes[5]
sensors = get_api('/'.join([data_url,site,node]))
sensor = sensors[3]
methods = get_api('/'.join([data_url,site,node,sensor]))
method = methods[1]
data = get_api('/'.join([data_url,site,node,sensor,method]))
data

['ctdbp_cdef_instrument_recovered']

In [50]:
for sensor in sensors:
    if 'CTDBP' in sensor:
        print(sensor)

03-CTDBPC000


In [55]:
# List the site, node, instrument names
site = 'CP01CNSM'
node = 'RID27'
sensor = '03-CTDBPC000'
method = 'recovered_host' # 'recovered_inst' 'telemetered'

In [56]:
# Retrieve vocabulary information for a given instrument
request_url = '/'.join((vocab_url, site, node, sensor))
r = requests.get(request_url, auth=(username, token))
data = r.json()

In [59]:
data[0]['refdes']

'CP01CNSM-RID27-03-CTDBPC000'

In [75]:
request_url = '/'.join((data_url, site, node, sensor, 'recovered_inst', 'ctdbp_cdef_instrument_recovered'))
r = requests.get(request_url, auth=(username, token))
data = r.json()
data

{'requestUUID': '233b711a-298c-43a5-a4e3-9fefca56d1f6',
 'outputURL': 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/areed@whoi.edu/20190315T193817-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/catalog.html',
 'allURLs': ['https://opendap.oceanobservatories.org/thredds/catalog/ooi/areed@whoi.edu/20190315T193817-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/catalog.html',
  'https://opendap.oceanobservatories.org/async_results/areed@whoi.edu/20190315T193817-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered'],
 'sizeCalculation': 63735283,
 'timeCalculation': 60,
 'numberOfSubJobs': 801}

In [ ]:
sen